## A.R.D.E 🔥🔥🔥

Integrantes:
- Agustín Pérez
- Rayén Godoy
- Dan Salgado
- Esperanza Oliva

### Contexto y motivación

El proyecto busca acercar el conocimiento sobre los exoplanetas , planetas que orbitan estrellas fuera de nuestro sistema solar, a niños y adolescentes, mediante la creación de materiales didácticos, visuales e interactivos basados en datos reales obtenidos por la página oficial:

    * URL: https://exoplanetarchive.ipac.caltech.edu/
            https://www.jpl.nasa.gov/edu/

En los últimos años, el número de exoplanetas descubiertos ha aumentado exponencialmente, y su estudio abre nuevas preguntas sobre la habitabilidad, la composición y la diversidad de mundos en nuestra galaxia. Sin embargo, gran parte de esta información se encuentra en bases de datos científicas complejas, poco accesibles para el público general.

Este proyecto tiene como motivación transformar datos científicos en conocimiento comprensible, fomentando la curiosidad de la astronomía desde edades tempranas.

### Preguntas objetivo

* ¿Qué sistemas planetarios tienen más planetas detectados?
    * metalicidad

* ¿Cómo ha evolucionado el número de descubrimientos de exoplanetas a lo largo de los años?


* ¿Cuál es la distancia promedio de los exoplanetas respecto a la Tierra según su tipo?

* ¿Existen diferencias notables en el tamaño o tipo de exoplanetas?

* ¿Que exoplanetas poseen características o condiciones similares a la tierra?



### Analisis exploratorio de datos

In [36]:
import pandas as pd
import numpy as np
import seaborn as sb
from os.path import join

path = join('..', 'data', 'PS_2025.10.17_07.44.21.csv')
df = pd.read_csv(path, skiprows=292, dtype={'hd_name': str, 'hip_name': str}).set_index('rowid')
df.head(3)

,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_dr2_id,gaia_dr3_id,default_flag,sy_snum,...,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec
rowid,,,,,,,,,,,,,,,,,,,,,
1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,Gaia DR3 3946945413106333696,1,2,...,2023-09-19,2023-08,2023-09-19,2,1,2,0,0,0,0
2,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,Gaia DR3 3946945413106333696,0,2,...,2014-07-23,2011-08,2014-07-23,2,1,2,0,0,0,0
3,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,Gaia DR3 3946945413106333696,0,2,...,2014-05-14,2008-01,2014-05-14,2,1,2,0,0,0,0


#### De momento para la exploración de los Datos, hemos agrupado algunas de las columnas más relevantes en 2 conjuntos principales que se muestran a continuación. Para las cuales se desglosan los siguientes DataFrames:


In [37]:
# DATAFRAME 1: Nos permite acceder a la información que se tiene de las características principales de cada exoplaneta 
# descubierto hasta la fecha, donde cada fila representa un nuevo registro de los datos asociados a un exoplaneta, el cual 
# puede ser una actualización de un planeta detectado anteriormente, o un registro de un nuevo exoplaneta. El significado 
# de las columnas se muestra como sigue:

# Datos del exoplaneta
hostname = 'hostname'  # Nombre de la estrella principal
pl_name = 'pl_name'  # Nombre del exoplaneta

# Datos orbitales
pl_orbper = 'pl_orbper'  # Periodo orbital (días)
pl_orbsmax = 'pl_orbsmax'  # Semieje mayor (UA), determina distancia media a la estrella

# Datos físicos del planeta
pl_rade = 'pl_rade'  # Radio del planeta (en tierras)
pl_masse = 'pl_masse'  # Masa del planeta (en tierras)
pl_dens = 'pl_dens'  # Densidad del planeta (g/cm^3), indica composición (rocoso/gaseoso)

# Datos de temperatura
pl_orbeccen = 'pl_orbeccen'  # Excentricidad orbital, mide estabilidad de temperatura
pl_eqt = 'pl_eqt'  # Temperatura de equilibrio (K), aproxima la temperatura superficial
pl_insol = 'pl_insol'  # Flujo de insolación [Tierra=1], indica energía recibida del sol

# Datos de la estrella (afectan energía recibida)
st_teff = 'st_teff'  # Temperatura efectiva estelar (K)
st_rad = 'st_rad'  # Radio de la estrella (en radios solares)
st_lum = 'st_lum'  # Luminosidad estelar (en log(Solar))
st_mass = 'st_mass'  # Masa de la estrella (en masas solares)

# Contexto observacional
sy_dist = 'sy_dist'  # Distancia al sistema (parsecs, distancia a la Tierra creo)
sy_snum = 'sy_snum'  # Número de estrellas en el sistema (1=solitario, >1=sistema múltiple)


In [38]:
# Dataframe con las columnas seleccionadas
df_exoplanets = df[[hostname, pl_name,
                    pl_orbper, pl_orbsmax,
                    pl_rade, pl_masse, pl_dens,
                    pl_orbeccen, pl_eqt, pl_insol,
                    st_teff, st_rad, st_lum, st_mass,
                    sy_dist, sy_snum]]

df_exoplanets.head(7)

,hostname,pl_name,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_dens,pl_orbeccen,pl_eqt,pl_insol,st_teff,st_rad,st_lum,st_mass,sy_dist,sy_snum
rowid,,,,,,,,,,,,,,,,
1,11 Com,11 Com b,323.21000,1.178,NaN,NaN,NaN,0.238,NaN,NaN,4874.0,13.76,1.97823,2.09,93.1846,2
2,11 Com,11 Com b,NaN,1.210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.60,93.1846,2
3,11 Com,11 Com b,326.03000,1.290,NaN,NaN,NaN,0.231,NaN,NaN,4742.0,19.00,2.24300,2.70,93.1846,2
4,11 UMi,11 UMi b,NaN,1.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.70,125.3210,1
5,11 UMi,11 UMi b,516.22000,1.540,NaN,NaN,NaN,0.080,NaN,NaN,4340.0,24.08,NaN,1.80,125.3210,1
6,11 UMi,11 UMi b,516.21997,1.530,NaN,NaN,NaN,0.080,NaN,NaN,4213.0,29.79,NaN,2.78,125.3210,1
7,14 And,14 And b,185.84000,0.830,NaN,NaN,NaN,0.000,NaN,NaN,4813.0,11.00,1.76300,2.20,75.4392,1


In [39]:
# DATAFRAME 2: Este conjunto de columnas se extrae con el objetivo de analizar factores externos que podrían incidir en
# el patrón de descubrimiento de los exoplanetas tomando como reerencia la fecha. Entre las columnas siguientes se consideran
# factores como el método de descubrimiento, lugar, telescopio utilizado, y estado del descubrimiento entre otros:

# Método de descubrimiento y contexto
discoverymethod = 'discoverymethod'  # Método de descubrimiento
discovery_year = 'disc_year'  # Año de descubrimiento
disc_refname = 'disc_refname'  # Referencia del descubrimiento
disc_pubdate = 'disc_pubdate'  # Fecha de publicación del descubrimiento
disc_locale = 'disc_locale'  # Lugar del descubrimiento
disc_facility = 'disc_facility'  # Instalación del descubrimiento
disc_telescope = 'disc_telescope'  # Telescopio del descubrimiento
disc_instrument = 'disc_instrument'  # Instrumento del descubrimiento

# Métodos de detección (flags)
rv_flag = 'rv_flag'  # Detectado por variaciones de velocidad radial
pul_flag = 'pul_flag'  # Detectado por variaciones de tiempo de púlsar
ptv_flag = 'ptv_flag'  # Detectado por variaciones de tiempo de pulsación
tran_flag = 'tran_flag'  # Detectado por tránsitos
ast_flag = 'ast_flag'  # Detectado por variaciones astrométricas
obm_flag = 'obm_flag'  # Detectado por modulaciones de brillo orbital
micro_flag = 'micro_flag'  # Detectado por microlente
etv_flag = 'etv_flag'  # Detectado por variaciones de tiempo de eclipses
ima_flag = 'ima_flag'  # Detectado por imágenes
dkin_flag = 'dkin_flag'  # Detectado por cinemática de disco

soltype = 'soltype'  # Tipo de solución (confirmado, candidato, falso positivo)


In [40]:
df_discovery = df[[pl_name, discoverymethod, discovery_year,
                   disc_refname, disc_pubdate, disc_locale,
                   disc_facility, disc_telescope, disc_instrument,
                   rv_flag, pul_flag, ptv_flag, tran_flag,
                   ast_flag, obm_flag, micro_flag, etv_flag,
                   ima_flag, dkin_flag, soltype]]

df_discovery.sort_values(by='disc_year', ascending=True, inplace=True)
df_discovery.head(7)

C:\Users\rayen\AppData\Local\Temp\ipykernel_14200\2600385485.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_discovery.sort_values(by='disc_year', ascending=True, inplace=True)


,pl_name,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype
rowid,,,,,,,,,,,,,,,,,,,,
35659,PSR B1257+12 c,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35660,PSR B1257+12 c,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35662,PSR B1257+12 d,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35661,PSR B1257+12 c,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35664,PSR B1257+12 d,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35663,PSR B1257+12 d,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35657,PSR B1257+12 b,Pulsar Timing,1994,<a refstr=WOLSZCZAN_1994 href=https://ui.adsab...,1994-04,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed


In [ ]:
discovery_time = df_discovery.drop_duplicates(subset= ['pl_name'], keep='last').sort_values(by= ['disc_year', 'discoverymethod'], ascending=True)
discovery_time
#Mi idea es con este df plotear un grafico de barras que en el eje x tenga el año de descubrimiento, y que el eje y sea la 
#cantidad de planetas descubiertos por año, pero clasificados por método de descubrimiento. Se me ocurre que podría ser un gráfico de barras
#multiples y que cada barra tenga un color según el método de descubrimiento.

,pl_name,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype
rowid,,,,,,,,,,,,,,,,,,,,
35661,PSR B1257+12 c,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35663,PSR B1257+12 d,Pulsar Timing,1992,<a refstr=WOLSZCZAN_ET_AL__1992 href=https://u...,1992-01,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
35658,PSR B1257+12 b,Pulsar Timing,1994,<a refstr=WOLSZCZAN_1994 href=https://ui.adsab...,1994-04,Ground,Arecibo Observatory,305 m Arecibo Radio Telescope,Arecibo Radio Reciever,0,1,0,0,0,0,0,0,0,0,Published Confirmed
71,51 Peg b,Radial Velocity,1995,<a refstr=MAYOR___QUELOZ_1995 href=https://ui....,1995-11,Ground,Haute-Provence Observatory,1.93 m Telescope,ELODIE Spectrograph,1,0,0,0,0,0,0,0,0,0,Published Confirmed
23,16 Cyg B b,Radial Velocity,1996,<a refstr=COCHRAN_ET_AL__1997 href=https://ui....,1997-07,Ground,Multiple Observatories,Multiple Telescopes,Multiple Instruments,1,0,0,0,0,0,0,0,0,0,Published Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35953,TOI-1266 d,Transit Timing Variations,2025,<a refstr=GREKLEK_MCKEON_ET_AL__2024 href=http...,2024-09,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,1,0,0,0,0,0,0,0,0,0,Published Confirmed
6425,KOI-134 c,Transit Timing Variations,2025,<a refstr=NABBIE_ET_AL__2025 href=https://ui.a...,2025-06,Space,Kepler,0.95 m Kepler Telescope,Kepler CCD Array,0,0,0,0,0,0,0,0,0,0,Published Confirmed
12396,Kepler-139 f,Transit Timing Variations,2025,<a refstr=LAMMERS__AMP__WINN_2025 href=https:/...,2025-04,Ground,Multiple Observatories,Multiple Telescopes,Multiple Instruments,1,0,0,0,0,0,0,0,0,0,Published Confirmed


### Datos

Los datos se obtienen de la URL: https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS usando TAP , descargando los datos de la tabla en formato CSV. Son datos estructurados, compuestos por:

* Tipos de datos :
    * float64(235), int64(25), object(28)
* 38973 filas, que corresponden a 38973 entradas de exoplanetas (posiblemente de un mismo planeta algunas ya que esta informacion es obtenida atravez de diferentes misiones, en las cuales se pudo haber divisado el mismo planeta)
* 288 columnas, que corresponden a los atributos encontrados de cada exoplaneta